<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/03-Regular_Expressions/A-Regular_Expressions_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regular Expressions




Regular expressions (regexes or re’s) constitute an extremely powerful, flexible and concise language for matching elements in text ranging from a few characters to complex patterns. While mastering the syntax of the regular expression language does require climbing a learning curve, this learning curve is not particularly steep, and a newcomer can find herself performing useful tasks with regular expressions almost immediately. Efforts spent learning regular expressions quickly pay off--tasks that are well suited for regular expressions abound. Indeed, regular expressions are one of the most useful computer skills, and an absolutely critical tool for data scientists. 

This document will present basic regular expression syntax and cover common use cases for regular expressions. 

In [1]:
# The code below is written in Python to replicate the 
# behavior of grep, the UNIX utility
# We will examine the details of how the code works in a subsequent notebook.
# For now, just execute the code, and use the function 
# grep(regex_expression, name_list) as-is

import re

def printMatches(text, regex_expression):
  BACKGROUND_YELLOW = '\x1b[43m'
  COLOR_RESET  = "\x1b[0m"
  regex= re.compile(regex_expression)
  matches = regex.finditer(text)
  for m in matches:
    highlighted  = text[:m.start()] # the string before the regex match
    highlighted += BACKGROUND_YELLOW + text[m.start():m.end()] + COLOR_RESET 
    highlighted += text[m.end():] # the string after the regex match
    print(highlighted)

def grep(regex_expression, name_list):
  for line in name_list:
    printMatches(line, regex_expression)

### NYC Restaurant Names Data

In the notebook, we will demonstrating the various regular expressions using the set of restaurant names from `/data/uniquenames.txt`.

In [2]:
!pip install -U -q PyMySQL sqlalchemy

from sqlalchemy import create_engine
import pandas as pd

conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    password = 'dwdstudent2015', 
    db = 'doh_restaurants',
    encoding = 'utf8mb4')

with create_engine(conn_string).connect() as mysql_conn:
  # This query returns back the restaurants in the DOH database
  sql = 'SELECT DISTINCT UPPER(DBA) AS DBA FROM restaurants WHERE DBA IS NOT NULL'
  uniquenames = pd.read_sql(sql, con=mysql_conn)
  uniquenames = uniquenames.DBA.values

print(f"Found {len(uniquenames)} unique restaurants names")

Found 14851 unique restaurants names


Let's take a peek at the entries of the `uniquenames` list:

In [3]:
uniquenames[:5]

array(['TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST',
       "WENDY'S", 'DJ REYNOLDS PUB AND RESTAURANT', 'RIVIERA CATERERS',
       "WILKEN'S FINE FOOD"], dtype=object)

In [4]:
uniquenames[-5:]

array(['PHO HOANG', 'CHICK ROCKS', 'NEW DOUBLE DRAGON',
       'ADONAY RESTAURANT', 'OH! BAGEL'], dtype=object)

Now, let's see if there are any restaurants with the string 'PANO' in them:

In [5]:
grep('PANO', uniquenames)

LA CANDELA ESPANOLA
PANORAMA OF MY SILENCE-HEART
GOOGLE PANORAMA
CENTRO ESPANOL
PANORAMA STEAKHOUSE
SABOR HISPANO
PANORAMA


What can we do if we want to search for something more complex than a fixed string? Regular expressions are solving exactly this problem. 

### The atoms

The simplest regular expressions are a sequence of `atoms`. An atom can be any of the following:
* single character, 
* a dot,
* a bracket expression, 
* an anchor.

#### Single character atom

A single character atom matches itself.

#### The `.` character atom

A dot atom matches any single character (except for a new line character `\n`).

Example: Using single character atoms, and the `.` atom, let's find all restaurant names that contain the characters `AB`, followed by any character (`.`) and then the character `D`:

In [6]:
grep('AB.D', uniquenames)

PABADE BAKERY & CAFE
CRAB DU JOUR XPRESS


#### Bracket expression atom

A bracket expression (defined by square brackets []) defines a set of characters. matches only one single character that can be any of the characters defined in a set. Example: [ABL] matches either A, B, or L.

Now, let's use a bracket expression: We want to find restaurants that contain one of the letters A,B,C,X,Y,Z followed by a digit. We specify the set of letters as `[ABCXYZ]` and the set of digits as `[0123456789]`.  

In [31]:
grep('[ABCXYZ][0123456789]', uniquenames)

B66 CLUB
B2 HARLEM
WORLD BEAN, VELOCITY BAR (EC2)
BARCLAYS UPPER SUITE STOLI BAR AND STORAGE ROOM 5C29.03
COTTO MARKET-GATE C30
GARDEN MARKET, STREET/HEALTH (FA6090)
BAR AT THE GARDEN (BA6110)
EVENT LEVEL CLUB (DELTA SKY 360 BAR) BA5075
HOT DOG CONCESSION (FC6100)
FOUR SEASONS HOTEL EMPLOYEE CAFETERIA4
A1 JAMAICA BREEZE
SIX2 RESTAURANT & BAKERY
SKY55 BAR AND GRILL


##### Brackets and ranges

Instead of typing long lists of characters in a bracket expression, we can use the range character: [0-9] is equivalent to [0123456789]. Similarly [A-Z] is equivalent to [ABCDEFGHIJKLMNOPQRSTUVWXYZ]. And [D-T] is equivalent to [DEFGHIJKLMNOPQRST]. (You get the idea.) You can also combine multiple ranges: [a-e1-9] is equivalent to [abcde123456789]. Finally, you can even specify to be excluded from the set using the character (^). For example, [^0-9] matches any character other than a number.

For example, let's find restaurants that contain a letter, followed by a number, and then followed by a charather that is not a number:

In [8]:
grep('[A-Z][0-9][^0-9]', uniquenames)

B2 HARLEM
WORLD BEAN, VELOCITY BAR (EC2)
TWO8TWO BAR & BURGER
BT4 AISLE B
BT3 AISLE A
OPPA , W4 PIZZA
A1 JAMAICA BREEZE
SIX2 RESTAURANT & BAKERY
TEAMWORKON3 JUICES
O2 K-BBQ


Hm, we do not want to get results that have a space after the number, so let's also exclude the space character:

In [9]:
grep('[A-Z][0-9][^0-9 ]', uniquenames) 

WORLD BEAN, VELOCITY BAR (EC2)
TWO8TWO BAR & BURGER


In [10]:
# Digit, not letter not digit not space, digit
grep('[0-9][^A-Z0-9 ][0-9]', uniquenames) 

104-01 FOSTER AVENUE COFFEE SHOP(UPS)
SUNSWICK 35/35
THE BEST $1.00 PIZZA
BARCLAYS UPPER SUITE STOLI BAR AND STORAGE ROOM 5C29.03
1.5 DAK GALBI
CHAIKHANA 7:40


In [11]:
# Restaurants with five digits
grep('[0-9][0-9][0-9][0-9][0-9]', uniquenames) 

STARBUCKS COFFEE # 26528
STARBUCKS COFFEE #22716
STARBUCKS COFFEE #23591
SUBWAY #47857
STARBUCKS COFFEE #29856
STARBUCKS COFFEE COMPANY #29897
STARBUCKS COFFEE #48170
STARBUCKS (STORE #50483)
STARBUCKS COFFEE #49952
STARBUCKS #50611
STARBUCKS COFFEE # 49196
STARBUCKS #48990
STARBUCKS COFFEE #49550
MCDONALD'S #11542
MCDONALD'S #13068
MCDONALD'S #23105
MCDONALDS # 18093
MCDONALDS 14520
SUBWAY (#29887)
STARBUCKS COFFEE #50622
SUBWAY STORE #30214
CARVEL # 102224
STARBUCKS COFFEE #29719
STARBUCKS COFFEE #53473
STARBUCKS COFFEE #49450
STARBUCKS #54446
STARBUCKS COFFE #55085
TACO BELL CANTINA 034646
STARBUCKS COFFEE#54771
CARVEL 102664
STARBUCKS COFFEE #52530
SUBWAY # 30658
TACO BELL # 35457
STARBUCKS COFFEE #58310
STARBUCKS COFFEE #56451
STARBUCKS COFFEE #57601
TACO BELL CANTINA #036012
STARBUCKS COFFEE #14090


#### Anchor

Anchor atoms are special characters, used to define the location of a regex within a line. 

The anchor `^` specifies the *beginning of a line*, the anchor `$` specifies the end of a line. The anchor `\b` specifies the word boundary.

Example: Find restaurant names that start with the characters `BAL`

In [12]:
grep('^BAL', uniquenames)

BALTHAZAR RESTAURANT
BALTHAZAR BAKERY
BALLATO'S RESTAURANT
BALDOR SPECIALTY FOODS
BALBOA RESTAURANT
BALADE
BALIMAYA RESTAURANT
BALABOOSTA
BALZEM
BALVANERA
BALADE EASTERN MEDITERRANEAN
BALANCERO


Example: Find restaurant names that end with the characters `SQUARE`

In [13]:
grep('SQUARE$', uniquenames)

MADISON SQUARE
MERRION SQUARE
TONIC TIMES SQUARE
NORTH SQUARE
HOLIDAY INN EXPRESS NYC TIMES SQUARE
HAMPTON INN-HERALD SQUARE
RESIDENCE INN TIMES SQUARE
THE MANHATTAN AT TIMES SQUARE
BEAN SQUARE
BEST WESTERN PREMIER HERALD SQUARE
HAMPTON INN TIMES SQUARE
ELEMENT HOTEL TIME SQUARE
TACOS TIME SQUARE
HOMEWOOD SUITES BY HILTON NEW YORK MIDTOWN MANHATTAN TIMES SQUARE
HOTEL RIU PLAZA NEW YORK TIMES SQUARE
MIAS BAKERY TIMES SQUARE
ICHIRAN TIMES SQUARE
PINNACLE BAGELS ON THE SQUARE


In [14]:
# All restaurants that end with 4 digits
grep('[0-9][0-9][0-9][0-9]$', uniquenames)

STARBUCKS #7277
STARBUCKS COFFEE #7358
STARBUCKS #7378
GALLAGHER'S 2000
CARVEL 2848
KAFFE 1668
THE ORIGINAL VINCENT'S ESTABLISH 1904
CHIPTOLE MEXICAN GRILL #2407
EVENT LEVEL CLUB (DELTA SKY 360 BAR) BA5075
SUITE 200, 1879
STARBUCKS COFFEE # 26528
STARBUCKS COFFEE #22716
STARBUCKS COFFEE #23591
LN 1380
SUBWAY #47857
CARVEL 1939
CHIPOTLE MEXICAN GRILL #2308
CHIPOTLE MEXICAN GRILL #2254
PANDA EXPRESS #2679
PANDA EXPRESS #2633
CHIPOTLE MEXCIAN GRILL # 2760
PANDA EXPRESS 2614
STARBUCKS COFFEE #29856
CHIPOTLE MEXICAN GRILL #2834
MEXICO 2000
CHIPOTLE MEXICAN GRILL #2838
STARBUCKS COFFEE COMPANY #29897
NEW GREAT WALL 1419
CHIPOTLE MEXICAN GRILL #2570
CHIPOTLE MEXICAN GRILL #2879
STARBUCKS COFFEE #48170
STARBUCKS COFFEE #49952
STARBUCKS #50611
STARBUCKS COFFEE # 49196
STARBUCKS #48990
STARBUCKS COFFEE #49550
MCDONALD'S #3880
MCDONALD'S #11542
MCDONALD'S #13068
MCDONALD'S #23105
MCDONALDS # 18093
MCDONALDS 14520
CHIPOTLE MEXICAN GRILL#2964
CHIPOTLE MEXICAN GRILL #3020
PANDA EXPRESS #2769
STARBUC

Example: Let's try to find restaurants containing the word `MEXICO`:

In [15]:
# Note that we capture also words like 'TULCIMEXICO' and 'MEXICOCIANA' 
grep('MEXICO', uniquenames)

MEXICO LINDO RESTAURANT
PIAXTLA ES MEXICO DELI
TACOS MEXICO
NEW MEXICO PLACE
SABOR A MEXICO II
EL SOL DE MEXICO DELI GROCERY
LAS MARAVILLAS DE MEXICO RESTAURANT
NUEVO MEXICO MEXICAN RESTAURANT
SABOR A MEXICO TAQUERIA
TACOS Y QUESADILLAS MEXICO
TAQUITOS MEXICO RESTAURANT
MADE IN MEXICO
MEXICO EN LA PIEL
MEXICO EL SALVADOR INC
MANJARES MEXICO
MEXICO 2000
VIVA MEXICO MEXICAN CUISINE
MEXICOCIANA
CON SABOR A MEXICO
LA ADELITA, EL CORAZON DE MEXICO
TULCIMEXICO RESTAURANT
EL RINCON DE MEXICO


In [16]:
# Notice that adding space is not sufficient
grep(' MEXICO ', uniquenames)

PIAXTLA ES MEXICO DELI
NEW MEXICO PLACE
SABOR A MEXICO II
EL SOL DE MEXICO DELI GROCERY
LAS MARAVILLAS DE MEXICO RESTAURANT
NUEVO MEXICO MEXICAN RESTAURANT
SABOR A MEXICO TAQUERIA
TAQUITOS MEXICO RESTAURANT
VIVA MEXICO MEXICAN CUISINE


Note that we also get TULCIMEXICO, MEXICOCIANA, which we _may_ not want. If we want only the word `MEXICO`, we add the word anchors:

In [17]:
# The r'....' is a "raw" string, and allows us to enter
# backslash without having to "escape" the backslash.
# Otherwise Python will interpret \b as a single special
# character, and not as two characters \b that are part of the regex
grep(r'\bMEXICO\b', uniquenames)

MEXICO LINDO RESTAURANT
PIAXTLA ES MEXICO DELI
TACOS MEXICO
NEW MEXICO PLACE
SABOR A MEXICO II
EL SOL DE MEXICO DELI GROCERY
LAS MARAVILLAS DE MEXICO RESTAURANT
NUEVO MEXICO MEXICAN RESTAURANT
SABOR A MEXICO TAQUERIA
TACOS Y QUESADILLAS MEXICO
TAQUITOS MEXICO RESTAURANT
MADE IN MEXICO
MEXICO EN LA PIEL
MEXICO EL SALVADOR INC
MANJARES MEXICO
MEXICO 2000
VIVA MEXICO MEXICAN CUISINE
CON SABOR A MEXICO
LA ADELITA, EL CORAZON DE MEXICO
EL RINCON DE MEXICO


#### Basic Patterns

* `a, X, 9, ....`: -- ordinary characters just match themselves exactly. 
* `. ^ \$ * + ? { [ ] \ | ( )`: The **meta-characters** which do not match themselves because they have special meanings (more info below)
* `.` (a period) -- matches any single character except newline '\n'
* `\t, \n, \r`: Special characters, tab, newline, return
* `^` = start, `$` = end -- match the start or end of the string
* `\`: inhibit the "specialness" of a character. So, for example, use \. to match a period or \\ to match a slash. If you are unsure if a character has special meaning, such as '@', you can put a slash in front of it, \@, to make sure it is treated just as a character.

#### Shortcuts

A few of the bracket expressions that we discussed above occur very often. For this reason, we have shortcuts for them:

* `\d`: matches the digits: `[0-9]`.
* `\D`: matches anything but `\d`: `[^0-9]`.
* `\w`: matches any alphanumeric character plus underscore: `[A-Za-z0-9_]`.
* `\W`: matches anything but `\w`: `[^A-Za-z0-9_]`
* `\s`: matches any "whitespace" character (space, tab, newline, etc): `[ \t\n\r\f\v]`.
* `\S`: matches anything but `\s`: `[^ \t\n\r\f\v]` .
* `\b`: matches the breaks between alphanumeric and non-alphanumeric characters (an empty string), the boundary between `\w` and `\W`. Useful for ensuring that what you match is actually a word.
* `\B`: matches anything but `\b`. Useful for ensuring your match is in the middle of a word.



#### In class exercises

Write a regular expression for:

* Match any character
* Match the end of line
* Match any digit
* Find all characters that are not digits
* Find all words with four letters
* Find every line that starts with a digit
* Find all empty lines
* Find all lines with 4 characters


### Regular Expressions: Operators

#### Alternation |

The alternation operator `|` defines one or more alternatives regular expressions that need to be true for the string to match the regular expression. 

For example, if we are looking for names that contain either the word `GREEK` or the word `RUSSIAN`, we issue the following command: 

In [18]:
grep('GREEK|RUSSIAN|FRENCH', uniquenames)

SYMPOSIUM GREEK RESTAURANT
RUSSIAN TURKISH BATHS
RUSSIAN SAMOVAR
FRENCH ROAST
RUSSIAN VODKA ROOM
JEAN DANET FRENCH PASTRY
THE GREEK KITCHEN
RUSSIAN BATHS
GREEK ISLANDS
AVLI THE LITTLE GREEK TAVERN
GREEK EXPRESS
SOMETHINGREEK
VILLAGE TAVERNA GREEK GRILL
JEAN CLAUDE FRENCH BISTRO
GRK FRESH GREEK
AVLEE GREEK KITCHEN
GREEK XPRESS
FRENCH LOUIE
FRENCH DINER
DIRTY FRENCH
MARATHI GREEK BISTRO
GREEK GRILL
GREEK EATS
EXCUSE MY FRENCH
3 GREEKS GRILL
AVLI LITTLE GREEK KAFE
KUZINA THE GREEK KITCHEN
GREEK FELLAS
LE FRENCH TART DELI
FRENCHETTE
FRENCHY COFFEE NYC
EONS GREEK FOOD FOR LIFE
GREEK FROM GREECE
LALGEROISE FRENCH BAKERY
SIMPLY GREEK


#### Repetition {m,n}

A repetition operator specifies that the atom or expression immediately before the repetition may be repeated. For example, if we are looking for restaurants that contain the letter I, three to five times:  

In [19]:
grep('I{3,5}', uniquenames)

EL CHIVITO D'ORO III
KNAPP PIZZA III
BARZOLA'S RESTAURANT III
LOS POLLITOS III
NEW WIN HING III CHINESE RESTAURANT
BAGEL EXPRESS III
LITTLE ITALY PIZZA III
ROCCO PIZZA III
MIRACALI III
EL POLLO III
CESTRAS PIZZA III
CHINA WOK III
EL NUEVO VALLE III
PHO RAINBOW III
RICO POLLO III
AVOCADO SUSHI IIII
PHO BEST III


Now, let's find all the restaurants that have a name length from 50 to 55 characters:

In [20]:
grep('^.{50,55}$', uniquenames)

VINNY'S OF CARROLL GARDEN RESTAURANT & LUNCHEONETT
IFH EL BUFFET RESTAURANT | ALBERTO'S MOFONGO HOUSE
MARRIOTT MARQUIS - MAIN KITCHEN/5TH FLOOR EMPLYEE CAFE
ADVENTURES AMUSEMENTS PARK (ICE CREAM, SWEETS STAND)
GREEN AND ACKERMAN KOSHER DAIRY RESTAURANT & PIZZA
ST JOHN'S UNIVERSITY LIBRARY CAFE  (ST.AUGUSTINE HALL)
CARIBBEAN CONNECTION CATERING SERVICES INC RESTAURANT
TAKE AWAY CAFE IN REBEKAH REHAB EXTENDED CARE CENTER
VISTA SKY LOUNGE & CATERING (SHERATON FOUR POINTS)
RESORTS WORLD CASINO GROUND LEVEL ( EMPLOYEE DINING)
THE BIG APPLE (RITZ CARLTON HOTEL EMPLOYEE CAFETERIA)
BARCLAYS LOWER SUITES TANDUAY BAR SOUTH CLUB LOUNGE
BARCLAYS UPPER SUITE STOLI BAR AND STORAGE ROOM 5C29.03
MANDARIN ORIENTAL NEW YORK- LOBBY LOUNGE 35TH FLOOR
THE BISTRO AT THE COURTYARD & RESIDENCE INN BY MARRIOTT
COLUMBIA UNIVERSITY BAKER ATHLETICS COMPLEX, STAND #1
COLUMBIA UNIVERSITY BAKER ATHLETICS COMPLEX, STAND #2
COLUMBIA UNIVERSITY BAKER ATHLETICS COMPLEX, STAND #3
MARKETPLACE AT MCGINLEY CENTER FORDHAM UN

In the repetition operator {m,n}, we can skip putting the upper limit if we want to say, "anything with m matches and above". For example, let's find all the restaurants that have a name length 60 characters and above:

In [21]:
grep('^.{60,}$', uniquenames)

TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST
SNACK BAR (LOCATED BETWEEN A-B BETWEEN FANCY FOOD AND MASTERS)
MAIMONIDES PARK STAND # 110, TIKI BAR/MR SOFTEE/MARTY'S BURGER
MAIMONIDES PARK STAND #120, PREMIO SAUSAGE/ARANCINI/CHEESESTEAKS/MR. SOFTEE
FASHION INSTITUTE OF TECHNOLOGY DAVID DUBINSKY STUDENT CENTER
WORLD ICE CAFE AT FLUSHING MEADOWS CORONA PARK AQUATIC CENTER
RED STORM DINER @ ST. VINCENT'S HALL OF ST. JOHN'S UNIVERSITY
DELTA SKY CLUB (BARTENDER SERVICE TERMINAL D DELTA DEPARTURE)
INTERCONTINENTAL NEW YORK TIMES SQUARE, TODD ENGLISH THE STINGER
MARLIN BAR AT TOMMY BAHAMA AND TOMMY BAHAMA RESTAURANT AND B
THE PENINSULA NEW YORK, CLEMENT/ THE GOTHAM LOUNGE/ SALON DE NING
UNION SQUARE SPORTS & ENTERTAINMENT AT THEATRE FOR A NEW AUDIENCE
FAIRFIELD INN & SUITES NEW YORK MANHATTAN FINANCIAL DISTRICT
MISTER DIPS (GARDEN AREA), LEUCA(1ST FLOOR) , WESTLIGHT (22ND FLOOR)
HOMEWOOD SUITES BY HILTON NEW YORK MIDTOWN MANHATTAN TIMES SQUARE
BARCEL

##### Repetition shortcuts (very common!): 

* `* = {0,}`. The `*` character means match the previous atom zero or more times
* `+ = {1,}`. The `+` character means match the previous atom one or more times
* `? = {0,1}`. The `*` character means match the previous atom zero or one times






Find all restaurants that start with one or more digits, followed by a space.

In [22]:
grep('^[0-9]+ ', uniquenames)

1 EAST 66TH STREET KITCHEN
21 CLUB
5 BURRO CAFE
3 GUYS
1020 BAR
810 DELI & CAFE
101 DELI
3 DELI & GRILL
15 EAST RESTAURANT
44 & X HELL'S KITCHEN
7 STARS RESTAURANT
3 SISTERS' & SHANTA'S RESTAURANT & BAKERY
68 JAY STREET BAR
3 STAR JUICE CENTER
3 WAY RESTAURANT
5 ESTRELLA BAKERY
169 BAR
809 GRILL & BAR RESTAURANT
230 FIFTH
11 STREET CAFE
234 CHINA CITY
27 SPORTS BAR & CAFE
535 MADISON CAFE
1 OAK
86 NOODLES
95 SOUTH
1 BANANA QUEEN
33 GOURMET
55 BAR
2 BROS PIZZA
67 ORANGE STREET
773 LOUNGE
18 BAKERY
1893 SPORTS BAR
212 HISAE'S
69 BAR LOUNGE
1 STOP PATTY SHOP
620 ON CATON
1001 NIGHTS CAFE
101 CAFE
5 NAPKIN BURGER
2 BROS PIZZA, CORVO COFFEE
18 RESTAURANT
888 KITCHEN
3 SHEETS SALOON
55 THOMAS CAFE
16 HANDLES
4618 BAKERY
6321 YUAN BAO
69 PIZZERIA
120 BAY CAFE/CARGO
68 MEI MEI BAKERY
718 HOOKAH LOUNGE
54 BELOW
983 BUSHWICK LIVING ROOM
5 STAR VILLAGE CAFE
12 CORAZONES RESTAURANT & BAR
88 CAFE
1803 NYC
99 FAVOR TASTE
401 LUCKY STAR RESTAURANT
624 KAM HAI CHINESE FOOD
212 STEAKHOUSE
7 GRAMS CAFFE

Find all restaurants that start with a letter, followed by one or more digits, followed by a space.

In [23]:
grep('^[A-Z][0-9]+ ', uniquenames)

B66 CLUB
B2 HARLEM
A1 JAMAICA BREEZE
O2 K-BBQ


In [24]:
# Find all restaurants
# Beggining with one or more letters // ^[A-Z]+
# followed by one or more digits // [0-9]+
# Followed by any number of charaters // .*
# and ending with BAR  // BAR$
grep('^[A-Z]+[0-9]+.*BAR$', uniquenames)

Find all restaurants that start with the word STARBUCKS, followed by any number of characters, and then have a digit.

In [25]:
grep('STARBUCKS.*[0-9]+', uniquenames)

STARBUCKS #847
STARBUCKS #7277
STARBUCKS COFFEE #7358
STARBUCKS #7378
STARBUCKS-ENTRANCE 11 WEST 33RD ST
STARBUCKS COFFEE # 26528
STARBUCKS COFFEE #22716
STARBUCKS COFFEE #23591
STARBUCKS COFFEE #29856
STARBUCKS COFFEE COMPANY #29897
STARBUCKS COFFEE #48170
STARBUCKS (STORE #50483)
STARBUCKS COFFEE #49952
STARBUCKS #50611
STARBUCKS COFFEE # 49196
STARBUCKS #48990
STARBUCKS COFFEE #49550
STARBUCKS COFFEE #50622
STARBUCKS COFFEE #29719
STARBUCKS COFFEE #53473
STARBUCKS COFFEE #49450
STARBUCKS #54446
STARBUCKS COFFEE #823
STARBUCKS COFFE #55085
STARBUCKS COFFEE#54771
STARBUCKS COFFEE #52530
STARBUCKS COFFEE #58310
STARBUCKS COFFEE #56451
STARBUCKS COFFEE #57601
STARBUCKS COFFEE #678 HUDSON
STARBUCKS COFFEE #14090


#### Grouping ()

In the group operator, when a group of characters is enclosed in parentheses, the next operator applies to the whole group, not only the previous characters. 

For example: Find all the restaurants that start (`^`) with 8 or more repetitions (`{8,}`) of the `\w+ ` pattern (alphanumeric characters followed by space):

In [26]:
grep(r'^(\w+ ){8,}', uniquenames)

TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST
WORLD ICE CAFE AT FLUSHING MEADOWS CORONA PARK AQUATIC CENTER
TAKE AWAY CAFE IN REBEKAH REHAB EXTENDED CARE CENTER
BARCLAYS UPPER SUITE STOLI BAR AND STORAGE ROOM 5C29.03
MARLIN BAR AT TOMMY BAHAMA AND TOMMY BAHAMA RESTAURANT AND B
HOMEWOOD SUITES BY HILTON NEW YORK MIDTOWN MANHATTAN TIMES SQUARE


#### In class exercices

What do these regular expressions match?

* b (cd)*
* h (d)+
* j? k+
* (cd){2,5}
* o(pre){3,}
* Panos|Ipeirotis

#### In class exercises (advanced)

Write down the regular expressions for the following:

* A telephone number (e.g, 212-555-0921)
* A zip+4 code (e.g, 10012-1809)
* For matching a float number (e.g., +12.34 or -1.457 or 1023.4568)
* Dollar amount with optional cents  (e.g. \$0.33, \$784)
* Time of Day (e.g. 12:15am, 3:34pm)
* Match urls  only of the form http://www.alphanumeric.com
* Match an email of the form username@domain (assume  that the domain might be in the form alphanumeric.alphanumeric, or alphanumeric.alphanumeric.alphanumeric)   



### Group references

Sometimes it is handy to be able to refer to a match that was made earlier in a regex. This is done with **backreferences**, which refer to groups. `\k` is the backreference specifier, where `k` is a number, which refers to the `k`-th regular expression *that was enclosed in parenthesis*.

For example, find if the first character(s) of a line are the same as the last:


In [27]:
grep(r'^(.{3,}).*\1$', uniquenames)

TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST
ZOO BREWS-BRONX ZOO
ARRIBA ARRIBA
PIO PIO
BARRACUDA BAR
KENNEDY FRIED CHICKEN
HARLEM BAR-B-Q, NABE HARLEM
SEBA SEBA
RUBY'S & LITTLE RUBY'S
ETCETERA ETCETERA
BARCELONA BAR
LAS RAMBLAS
SEBA-SEBA
TETE-A-TETE
ANTONIO'S RESTAURANT
LOS 3 POTRILLOS
PIO-PIO
ONE AND ONE
CHOP CHOP
BOULUD SUD & EPICERIE BOULUD
CHEEBURGER CHEEBURGER
GONZALEZ Y GONZALEZ
SHIMBASHI SUSHI
BARCLAYS NORTH SUITE STOLI BAR
BARCLAYS 40, 40 CLUB BAR
LOS TRES POTRILLOS
YUMMY YUMMY
BERON BERON
PIL PIL
VIS-A-VIS
CHEN'S KITCHEN
BARRY'S BOOTCAMP FUEL BAR
MANGO MANGO
TUK TUK
CHA CHA MATCHA
YIA YIA
KENNEDY CHICKEN
WIKIWIKI
BEAN N BEAN
PANADERIA IXTAPAN
BEAN & BEAN
MAKIMAKI
RICHIE RICH
TOFU TOFU
CITY ISLAND IN THE CITY
SANSOOKAPSAN
BAAR BAAR
PERI PERI
CACHAPA CACHAPA
KENNEDY'S FRIED CHICKEN
TETE A TETE
SUSHI SUSHI
LOS MARINILLOS
CHEN'S EXPRESS KITCHEN
SHIMA SUSHI
SUNDAY TO SUNDAY
TOMO TOMO


Or find all the restaurant names that the first 5 characters (or more) are identical to the last characters.

In [28]:
grep(r'^([A-Z]+)\1$', uniquenames)

NONO
MIMI
MEME
WIKIWIKI
MAKIMAKI
COCO
MAMA
ZIZI


Or find the restaurants where the first character is the same as the last, and the second character is the same as the penultimate character.

In [35]:
grep(r'^(.)(.).*\2\1$', uniquenames)

Find all names that have three consecutive same digits

In [29]:
grep(r'([0-9])\1\1', uniquenames)

GALLAGHER'S 2000
LEGENDS 000
888 KITCHEN
333 LOUNGE
MEXICO 2000
GOOD TASTE 666
CARVEL # 102224
STARBUCKS #54446
NEW HONG KONG 3888 INC
SHAXIAN 888
CHECKERS STORE #3332
CAFE 2000


As we are going to see, these backreferences will also be of tremendous use for extraction purposes.

In [30]:
#### Naming groups
# The group that follows the term "DOUBLE" is named "doublewhat" and we can refer to it as \doublewhat
grep(r'DOUBLE (?P<doublewhat>\w+)', uniquenames)


DOUBLE DOWN SALOON
DOUBLE DRAGON RESTAURANT
DOUBLE WINDSOR
NEW DOUBLE CHINESE RESTAURANT
DOUBLE RAINBOW
BEST DOUBLE DRAGON RESTAURANT
DOUBLE DUTCH ESPRESSO
DOUBLE TOP CHINA & TORTILLA TACO
DOUBLE DRAGON
DOUBLE DRAGON 88
DOUBLE ZERO
108 DOUBLE CHINESE RESTAURANT
DOUBLE BEN CAFE
DOUBLE CRISPY BAKERY I
A & A BAKE AND DOUBLE SHOP
DOUBLE RED INC
DOUBLE TWISTER ICE CREAM AND COFFEE SHOP
NEW DOUBLE DRAGON


#### In class exercise (advanced)

Say that you have a file with telephone numbers written in a variety of forms: 

* 679-397-5255
* 2126660921
* 212-998-0902
* 888-888-2222
* 800-555-1211
* 800 555 1212
* 800.555.1213
* (800) 555-1214
* 1-800-555-1215
* 1(800)555-1216
* 800-555-1212-1234
* 800-555-1212x1234
* 800-555-1212 ext. 1234
* work 1-(800) 555.1212 #1234

The task is to standardize everything in the form (xxx)-xxx-xxx.


To make the process interactive, go to http://regex101.com/?#python, copy and paste the numbers above in the textarea called "Text String", and then try to write the regular expression above. (As a side note, the website provides excellent explanations about the meaning of the regular expression that you write down.) Remember to put the `"g"` character in the small textfield next to the regex: that `g` means "find globally" the regex, not just the first occurence.

###  Exercise 2

Retrieve news articles using the NewsAPI. Identify articles that mention a dollar amount (i.e., the `$` symbol, followed by numbers)

### Additional Regex Resources

* [Visual Regular Expression Tester](http://www.debuggex.com/?flavor=pcre)
* [Test Python Regular Expressions Online](http://www.pyregex.com/)
* [Regular Expressions 101](http://regex101.com/?)
* [Python's re Library Official Documentation](http://docs.python.org/2/library/re.html)
* [Regular expression reference at W3schools](http://www.w3schools.com/jsref/jsref_obj_regexp.asp)
* [Parsing phone numbers using Python and regular expressions](http://www.diveintopython.net/regular_expressions/phone_numbers.html)

### More Advanced Regular Expressions

And the ones below get a little bit more advanced:

* `*?`, `+?`: ordinarily, `*`, `+` and `?` are **greedy**. This means they are matching the longest possible string that satisfies the regular expression. Adding the `?` to any of these makes it non-greedy, instead matching the shortest possible expression. 
* `(?: )`: A non-capturing group. This works just as `()`, but doesn’t hold on to the matched contents.
* `(?<=x)`: Matches any string that is preceded by x (an arbitrary regular expression).
